# Exercise 3.3

BEGIN: I wrote this code personally without assistance. Any fragments taken from external sources will be explicitly marked.

This algorithm is an improved version of the LSB audio steganography algorithm provided in Exercise 3.1. The original algorithm creates recognizable noise in the modified audio file. This version aims to address this issue.

To overcome this issue, I use a special message termination code, eliminating the need to modify all the last bits of all frames. As a result, only the small noisy section is confined to the beginning of the audio file, making it almost unnoticeable.

Additionally, I incorporated a non-consecutive pattern of bits to store the message. This further enhances the difficulty of detecting the embedded message and makes the noise even more unnoticeable.

In [7]:
import wave

Here is a special [End-of-transmission character](https://en.wikipedia.org/wiki/End-of-Transmission_character) to designate the end of the encoded string:

In [8]:
MESSAGE_TERM_CODE = "\u0004"

In [9]:
def hide_message(input_file, output_file, message):
    with wave.open(input_file, 'rb') as input_wave:
        # Convert audio file into bytes
        frame_bytes = bytearray(list(input_wave.readframes(input_wave.getnframes())))

        # add padding
        message += MESSAGE_TERM_CODE

        # convert secret to a list of bytes represented as a sting
        bytes_strings = [bin(b).lstrip('0b').rjust(8, '0') for b in bytes(message, "ascii")]
        bit_string = ''.join(bytes_strings)
        bits = list(map(int, bit_string))

        for i, bit in enumerate(bits):
            # We write our bytes not in a row, but using unequal intervals (i*10+i)
            # to eliminate any recognizable pattern
            frame_bytes[i*10+i] = (frame_bytes[i] & 254) | bit

        frame_modified = bytes(frame_bytes)

        #  
        with wave.open(output_file, 'wb') as output_wave:
            output_wave.setparams(input_wave.getparams())
            output_wave.writeframes(frame_modified)

In [10]:
hide_message('sounds/Ex3_sound5.wav', 'song_embedded.wav', "Father Christmas does not exist")

In [11]:
def unhide_message(input_file):
    with wave.open(input_file, 'rb') as input_wave:
        frame_bytes = bytearray(list(input_wave.readframes(input_wave.getnframes())))

        extracted = []
        for i in range(len(frame_bytes)):
            # We write our bytes not in a row, but using unequal intervals (i*10+i)
            # to eliminate any recognizable pattern
            secret_message_index = (i*10+i)
            if secret_message_index > len(frame_bytes)-1:
                break

            extracted.append(frame_bytes[secret_message_index] & 1)

        message = "".join(chr(int("".join(map(str, extracted[i:i+8])),2)) for i in range(0,len(extracted),8))

        return(message.split(MESSAGE_TERM_CODE)[0])

In [12]:
unhide_message('song_embedded.wav')

'Father Christmas does not exist'

END of my code